In [ ]:
# in this notebook varya is trying to export the median point (from the precipitation) from HMA - regions 
# to a dataframe and then later csv file to use as a forsing for testrun of sedcas model 

# take the locations determined in a different notebook (also stored in some csv file already
# find that location in the HMA climate from the server 
# convert to csv 
# save 


In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
import os

from datetime import datetime



In [ ]:
# read locations file - for annual sum precipitations: 
locations_pr = pd.read_table('out/median_annual_precip_regions.csv', sep = ',',index_col =0)

# read locations file - for annial median temp:
locations_t2m = pd.read_table('out/annual_median_t2m_regions.csv', sep = ',', index_col = 0)

# read hma polygons file to a df:
hma = gpd.read_file('HMA_regions/HMA_regions.shp')
hma_df = pd.DataFrame(hma) 

In [ ]:
# locations_pr.head()
# locations_t2m

In [ ]:
%%time
# ----------- read climate data: ----------- 

# temperatures
temps = xr.open_mfdataset("/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/2m-temperature/*.nc")

# precipitations
precip = xr.open_mfdataset("/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-precipitation/*.nc")

# total cloud cover 
clouds = xr.open_mfdataset("/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-cloud-cover/*.nc")

# incoming radiation 
radiation = xr.open_mfdataset("/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/surface-solar-radiation-downwards/*.nc")


# path for output timeseries
path_output = "/Users/varyabazilova/Desktop/hma_regions/out/timeseries"


In [ ]:
# len(temps.t2m.values)

In [ ]:
# ----------- convert units: ----------- 

# climate = xr.open_mfdataset('*.nc', decode_coords="all")
# cnange units (and overwrite the metadata with the units after)

# ------ UNITS ------

#convert temperature K to C
temps['t2m']=temps.t2m-273.15
# precipotation m to mm 
precip['tp']=precip.tp * 1000
# radiation j/m2 to w/m2
# SSR [W/m2] = SSR [J/m^2] / (3600 seconds)
radiation['ssrd'] = radiation.ssrd / 3600

### (1) select coordinates 

In [ ]:
# take the n index of the "locations precipitation" table 

n = 13
print('name of mnt range:', hma_df.Name[n])

point_lat = locations_pr.latitude[n]
point_lon = locations_pr.longitude[n]

In [ ]:
# %%time

# kick out unnecessary dimention:
temps = temps.sel(expver= slice(0, 1))
precip = precip.sel(expver = slice(0,1))
clouds = clouds.sel(expver = slice(0,1))
radiation = radiation.sel(expver = slice(0,1))

# select the point in the each dataset 

temps_point = temps.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
precip_point = precip.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
clouds_point = clouds.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
radiation_point = radiation.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')

# make a time slice so that it dpesnt take THAT much long: 
temps_point = temps_point.sel(time=slice("1979-01-01", "1980-01-01")).drop(labels = ['longitude', 'latitude']).squeeze()#.drop(label = 'expver')

# drop coordinates dimention: 
# temps_point = temps_point.drop(labels = ['longitude', 'latitude', 'expver'])

# temps_point1 = temps_point.squeeze()


In [ ]:
# temps_point

In [ ]:
%%time
df = temps_point.to_dataframe()#.dropna().drop_duplicates()

In [ ]:
# df

In [ ]:
%%time

# df = temps_point1.to_dataframe().dropna().drop_duplicates()

In [ ]:
%%time
temps_point1_df = temps_point1.to_dataframe().dropna().drop_duplicates()

In [ ]:
# %%time
# ----------- convert to dataframe and cleanup: ----------- 

# temps_point_df = temps_point.to_dataframe()

# temps_point1_df

In [ ]:
# temps_point
len(temps.time.values)

## workapathround (make netcdf conversion to df faster): 

### (this is an important part) 

In [3]:
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
import os

from datetime import datetime




In [4]:
# read locations file - for annual sum precipitations: 
locations_pr = pd.read_table('out/median_annual_precip_regions.csv', sep = ',',index_col =0)

# read locations file - for annial median temp:
locations_t2m = pd.read_table('out/annual_median_t2m_regions.csv', sep = ',', index_col = 0)

# read hma polygons file to a df:
hma = gpd.read_file('HMA_regions/HMA_regions.shp')
hma_df = pd.DataFrame(hma) 

In [5]:
path_temp = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/2m-temperature/'

# precipitations
path_precip = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-precipitation/'

# total cloud cover 
path_clouds = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/total-cloud-cover/'

# incoming radiation 
path_radiation = '/Volumes/Data/Repository/external_data/ERA5/HMA/netcdf/hourly/surface-solar-radiation-downwards/'


# path for output timeseries
path_output_temp = "/Users/varyabazilova/Desktop/hma_regions/out/timeseries"

In [10]:
# take the n index of the "locations precipitation" table 

n = 13
print('name of mnt range:', hma_df.Name[n])

point_lat = locations_pr.latitude[n]
point_lon = locations_pr.longitude[n]

# print(datetime.now())

name of mnt range: Tibetan Interior Mountains


In [8]:
'''%%time

# temperatures

print('started at:', datetime.now())

for n, f in enumerate(os.listdir(path_temp)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_temp, f), decode_coords="all")
    # select coordinates: 
    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
    # kick out the expver dimention
    # ds = ds.sel(expver= slice(0, 1))
    # ds = ds.squeeze()
    ds = ds.drop(labels = ['longitude', 'latitude']).squeeze()
    
    # check:
    print('ready to turn into df at:', datetime.now())
    
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna().drop_duplicates()
    
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    ds_df.to_csv(path_output_temp + '_temp_{y}.csv'.format(y=year))
    
    print('year - done:', year)
    print('done at:', datetime.now())
    
 '''


'%%time\n\n# temperatures\n\nprint(\'started at:\', datetime.now())\n\nfor n, f in enumerate(os.listdir(path_temp)):\n    # read every file\n    ds = xr.open_dataset(os.path.join(path_temp, f), decode_coords="all")\n    # select coordinates: \n    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = \'nearest\')\n    # kick out the expver dimention\n    # ds = ds.sel(expver= slice(0, 1))\n    # ds = ds.squeeze()\n    ds = ds.drop(labels = [\'longitude\', \'latitude\']).squeeze()\n    \n    # check:\n    print(\'ready to turn into df at:\', datetime.now())\n    \n    # convert to dataframe\n    ds_df = ds.to_dataframe().dropna().drop_duplicates()\n    \n    # export:\n    # 1. create a year \n    year = n + 1979\n    # 2. save to csv\n    ds_df.to_csv(path_output_temp + \'_temp_{y}.csv\'.format(y=year))\n    \n    print(\'year - done:\', year)\n    print(\'done at:\', datetime.now())\n    \n '

In [ ]:
## read all the csv files created 

# append from the bottom 


In [9]:
# path for output timeseries
path_output = "/Users/varyabazilova/Desktop/hma_regions/out/timeseries"

In [12]:
'''%%time

# precipitation 

print('started at:', datetime.now())

for n, f in enumerate(os.listdir(path_precip)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_precip, f), decode_coords="all")
    # select coordinates: 
    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
    # kick out the expver dimention
    # ds = ds.sel(expver= slice(0, 1))
    # ds = ds.squeeze()
    ds = ds.drop(labels = ['longitude', 'latitude']).squeeze()
    
    # check:
    print('ready to turn into df at:', datetime.now())
    
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna().drop_duplicates()
    
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    ds_df.to_csv(path_output + '_precip_{y}.csv'.format(y=year))
    
    print('year - done:', year)
    print('done at:', datetime.now())
    
''' 



'%%time\n\n# precipitation \n\nprint(\'started at:\', datetime.now())\n\nfor n, f in enumerate(os.listdir(path_precip)):\n    # read every file\n    ds = xr.open_dataset(os.path.join(path_precip, f), decode_coords="all")\n    # select coordinates: \n    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = \'nearest\')\n    # kick out the expver dimention\n    # ds = ds.sel(expver= slice(0, 1))\n    # ds = ds.squeeze()\n    ds = ds.drop(labels = [\'longitude\', \'latitude\']).squeeze()\n    \n    # check:\n    print(\'ready to turn into df at:\', datetime.now())\n    \n    # convert to dataframe\n    ds_df = ds.to_dataframe().dropna().drop_duplicates()\n    \n    # export:\n    # 1. create a year \n    year = n + 1979\n    # 2. save to csv\n    ds_df.to_csv(path_output + \'_precip_{y}.csv\'.format(y=year))\n    \n    print(\'year - done:\', year)\n    print(\'done at:\', datetime.now())\n    \n'

In [14]:
'''%%time

# clouds 

print('started at:', datetime.now())

for n, f in enumerate(os.listdir(path_clouds)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_clouds, f), decode_coords="all")
    # select coordinates: 
    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
    # kick out the expver dimention
    # ds = ds.sel(expver= slice(0, 1))
    # ds = ds.squeeze()
    ds = ds.drop(labels = ['longitude', 'latitude']).squeeze()
    
    # check:
    print('ready to turn into df at:', datetime.now())
    
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna().drop_duplicates()
    
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    ds_df.to_csv(path_output + '_cloud_cover_{y}.csv'.format(y=year))
    
    print('year - done:', year)
    print('done at:', datetime.now())
    
 '''



'%%time\n\n# clouds \n\nprint(\'started at:\', datetime.now())\n\nfor n, f in enumerate(os.listdir(path_clouds)):\n    # read every file\n    ds = xr.open_dataset(os.path.join(path_clouds, f), decode_coords="all")\n    # select coordinates: \n    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = \'nearest\')\n    # kick out the expver dimention\n    # ds = ds.sel(expver= slice(0, 1))\n    # ds = ds.squeeze()\n    ds = ds.drop(labels = [\'longitude\', \'latitude\']).squeeze()\n    \n    # check:\n    print(\'ready to turn into df at:\', datetime.now())\n    \n    # convert to dataframe\n    ds_df = ds.to_dataframe().dropna().drop_duplicates()\n    \n    # export:\n    # 1. create a year \n    year = n + 1979\n    # 2. save to csv\n    ds_df.to_csv(path_output + \'_cloud_cover_{y}.csv\'.format(y=year))\n    \n    print(\'year - done:\', year)\n    print(\'done at:\', datetime.now())\n    \n '

In [16]:
'''%%time

# radiation 

print('started at:', datetime.now())

for n, f in enumerate(os.listdir(path_radiation)):
    # read every file
    ds = xr.open_dataset(os.path.join(path_radiation, f), decode_coords="all")
    # select coordinates: 
    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = 'nearest')
    # kick out the expver dimention
    # ds = ds.sel(expver= slice(0, 1))
    # ds = ds.squeeze()
    ds = ds.drop(labels = ['longitude', 'latitude']).squeeze()
    
    # check:
    print('ready to turn into df at:', datetime.now())
    
    # convert to dataframe
    ds_df = ds.to_dataframe().dropna().drop_duplicates()
    
    # export:
    # 1. create a year 
    year = n + 1979
    # 2. save to csv
    ds_df.to_csv(path_output + '_SWradiation_{y}.csv'.format(y=year))
    
    print('year - done:', year)
    print('done at:', datetime.now())
    '''
 




'%%time\n\n# radiation \n\nprint(\'started at:\', datetime.now())\n\nfor n, f in enumerate(os.listdir(path_radiation)):\n    # read every file\n    ds = xr.open_dataset(os.path.join(path_radiation, f), decode_coords="all")\n    # select coordinates: \n    ds = ds.sel(latitude = point_lat, longitude = point_lon, method = \'nearest\')\n    # kick out the expver dimention\n    # ds = ds.sel(expver= slice(0, 1))\n    # ds = ds.squeeze()\n    ds = ds.drop(labels = [\'longitude\', \'latitude\']).squeeze()\n    \n    # check:\n    print(\'ready to turn into df at:\', datetime.now())\n    \n    # convert to dataframe\n    ds_df = ds.to_dataframe().dropna().drop_duplicates()\n    \n    # export:\n    # 1. create a year \n    year = n + 1979\n    # 2. save to csv\n    ds_df.to_csv(path_output + \'_SWradiation_{y}.csv\'.format(y=year))\n    \n    print(\'year - done:\', year)\n    print(\'done at:\', datetime.now())\n    '